# 5. Train

## 5.1 训练神经网络的步骤

### 5.1.1 训练神经网络的七个步骤
* 从训练集中获取批量数据
* 将批量数据传入网络
* 计算损失(预测值与真实值之间的差)【需要loss function实现】
* 计算损失函数的梯度  【需要back propagation实现】
* 通过上一步计算的梯度来更新权重，进而减少损失【需要optimization algorithm实现】
* 重复1-5步直到一个epoch执行完成
* 重复1-6步直到所设定的epochs执行完成并得到满意的accuracy

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)

print(torch.__version__)
print(torchvision.__version__)

1.6.0
0.7.0


In [2]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
        
    def forward(self, t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        
        t = F.relu(self.fc2(t))
        
        t = self.out(t)
        
        return t

In [9]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST/',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)

In [10]:
def get_num_corrent(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [11]:
network = Network()
optimizer = optim.Adam(network.parameters(), lr=0.01)

In [12]:
for epoch in range(2):
    total_loss = 0
    total_corrent = 0
    
    for batch in train_loader:
        images, labels = batch
        # 预测
        preds = network(images)
        # loss function
        loss = F.cross_entropy(preds, labels)
        # 计算梯度
        optimizer.zero_grad()
        loss.backward()
        # 更新权重
        optimizer.step()
        
        total_loss += loss.item()
        total_corrent += get_num_corrent(preds, labels)
    print(f'epoch: {epoch}, loss: {total_loss}, total_corrent: {total_corrent}')

epoch: 0, loss: 371.49561128020287, total_corrent: 45891
epoch: 1, loss: 243.01265758275986, total_corrent: 51091


In [13]:
accuracy = total_corrent / len(train_set)
accuracy

0.8515166666666667

* 每个周期的迭代数 = 数据总数/batchsize（当改变batchsize时，也就是改变了更新权重的次数，也就是朝损失函数最小的防线前进的步数）
* accuracy = total_correct/len(train_set)
* 梯度：告诉我们应该走哪条路能更快的到达loss最小